In [1]:
import pandas as pd
import pyodbc
import useful_functions as use

In [2]:
from fuzzywuzzy import fuzz

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
#iqvia
#dhc
#definitive
#symphony
#datagov

In [ ]:
#iqvia
username = 'vigrose'
password = 'Slytherin~10946'
q = "DSN=eprdods; UID={}; PWD={}".format(username, password)
ODS = pyodbc.connect(q)
iqvia_query = \
        """
        SELECT DISTINCT 
        P.ROLE,
        P.PRIMARY_SPEC,
        P.SECONDARY_SPEC,
        P.PRIMARY_PROF_DESC,
        P.STATUS_DESC,
        P.TERTIARY_SPEC,
        B.COT_CLASSIFICATION,
        B.COT_FACILITY_TYPE,
        B.COT_SPECIALTY,
        P.ME
        FROM 
        ODS.ODS_IMS_BUSINESS B, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.ODS_IMS_PROFESSIONAL P, ODS.ODS_IMS_AFFILIATION_TYPE T
        WHERE  
        B.IMS_ORG_ID = A.IMS_ORG_ID
        AND
        A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
        AND
        P.CURRENT_BATCH_FLAG='Y'
        AND
        A.CURRENT_BATCH_FLAG='Y'
        AND
        B.CURRENT_BATCH_FLAG='Y'
        """
iqvia = pd.read_sql(con=ODS, sql=iqvia_query)
iqvia.head()

In [6]:
dhc = pd.read_csv('../../Data/DHC/Practice.csv')

In [7]:
gov_file = use.get_newest('../../Data/DataGov/','DAC')
gov = pd.read_csv(gov_file, low_memory=False)

In [8]:
#symphony
sym_query = \
        """
        SELECT
        d.PRCTR_DS_RPTD_PRMRY_SPCLT_DESC,
        d.PRCTR_DS_RPTD_SCNRY_SPCLT_DESC,
        l.OTHER_ID AS SYM_ME
        FROM
        ODS.PRACTITIONER_DEMOGRAPHIC_LAYOUT d, ODS.PRACTITIONER_ADDL_IDS_LAYOUT l
        WHERE
        d.DS_PRCTR_ID = l.DS_PRCTR_ID
        and
        l.ID_QLFR_TYP_CDE = 38
        """  
symphony = pd.read_sql(con=ODS, sql=sym_query)
symphony.head()

,PRCTR_DS_RPTD_PRMRY_SPCLT_DESC,PRCTR_DS_RPTD_SCNRY_SPCLT_DESC,SYM_ME
0,"SURGERY, PLASTIC",None,0251279056
1,PEDIATRICS,None,0251277112
2,INTERNAL MEDICINE,None,0251277114
3,"SURGERY, GENERAL",None,0251277115
4,GERIATRIC MEDICINE,INTERNAL MEDICINE,0251277116


In [3]:
ppd = use.get_newest_ppd()

INFO:useful_functions:Newest ppd is from Aug 13 2022
INFO:useful_functions:Getting PPD data from UDrive...
INFO:useful_functions:Saving PPD as csv in ppd folder...


In [10]:
ppd['IQVIA_ME'] = [x[0:10] for x in ppd.ME]

In [12]:
HMM = pd.merge(dhc, gov, on='NPI')
HMM.ME = use.fix_me(HMM.ME)
HMM = pd.merge(HMM, ppd, on='ME')
HMM = HMM.drop_duplicates(['ME','sec_spec_all'])
HMM = pd.merge(HMM, iqvia, left_on='IQVIA_ME', right_on='ME', suffixes = ['','_IQVIA'], how = 'left')
HMM = pd.merge(HMM, symphony, left_on='IQVIA_ME', right_on='SYM_ME', suffixes = ['','_SYM'], how = 'left')

In [28]:
ppd.columns

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'SECTOR', 'CARRIER_ROUTE',
       'ADDRESS_UNDELIVERABLE_FLAG', 'FIPS_COUNTY', 'FIPS_STATE',
       'PRINTER_CONTROL_CODE', 'PC_ZIP', 'PC_SECTOR', 'DELIVERY_POINT_CODE',
       'CHECK_DIGIT', 'PRINTER_CONTROL_CODE_2', 'REGION', 'DIVISION', 'GROUP',
       'TRACT', 'SUFFIX_CENSUS', 'BLOCK_GROUP', 'MSA_POPULATION_SIZE',
       'MICRO_METRO_IND', 'CBSA', 'CBSA_DIV_IND', 'MD_DO_CODE', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 'GENDER',
       'TELEPHONE_NUMBER', 'PRESUMED_DEAD_FLAG', 'FAX_NUMBER', 'TOP_CD',
       'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD', 'PRA_RECIPIENT',
       'PRA_EXP_DT', 'GME_CONF_FLG', 'FROM_DT', 'TO_DT', 'YEAR_IN_PROGRAM',
       'POST_GRADUATE_YEAR', 'GME_SPEC_1', 'GME_SPEC_2', 'TRAINING_TYPE',
       'GME_INST_STATE', 'GME_INST_ID', 'ME

In [7]:
specs = pd.read_csv('../../Data/PPD/speciality_id.csv')

In [33]:
HMM[['pri_spec','Primary Specialty','PRIMARY_SPEC','PRCTR_DS_RPTD_PRMRY_SPCLT_DESC', 'DESC']]

,pri_spec,Primary Specialty,PRIMARY_SPEC,PRCTR_DS_RPTD_PRMRY_SPCLT_DESC,DESC
0,OPHTHALMOLOGY,Ophthalmology,OPHTHALMOLOGY,OPHTHALMOLOGY,OPHTHALMOLOGY
1,OPHTHALMOLOGY,Ophthalmology,OPHTHALMOLOGY,OPHTHALMOLOGY,OPHTHALMOLOGY
2,OPHTHALMOLOGY,Ophthalmology,OPHTHALMOLOGY,OPHTHALMOLOGY,OPHTHALMOLOGY
3,OPHTHALMOLOGY,Ophthalmology,OPHTHALMOLOGY,OPHTHALMOLOGY,OPHTHALMOLOGY
4,OPHTHALMOLOGY,Ophthalmology,OPHTHALMOLOGY,OPHTHALMOLOGY,OPHTHALMOLOGY
...,...,...,...,...,...
258718,EMERGENCY MEDICINE,Emergency Medicine,CLINICAL INFORMATICS (PATHOLOGY),EMERGENCY MEDICINE,CLINICAL INFORMATICS (PATHOLOGY)
258719,HOSPITALIST,Hospitalist,HOSPITALIST,INTERNAL MEDICINE,HOSPICE & PALLIATIVE MEDICINE (ANESTHESIOLOGY)
258720,HOSPITALIST,Hospitalist,HOSPITALIST,INTERNAL MEDICINE,HOSPICE & PALLIATIVE MEDICINE (ANESTHESIOLOGY)
258721,HOSPITALIST,Hospitalist,HOSPITALIST,INTERNAL MEDICINE,HOSPICE & PALLIATIVE MEDICINE (ANESTHESIOLOGY)


In [30]:
HMM = pd.merge(HMM, specs, left_on='PRIM_SPEC_CD', right_on='SPEC_CD')

In [32]:
specs.columns

Index(['SPEC_CD', 'SPEC_ID', 'DESC'], dtype='object')

In [40]:
HMM = HMM.drop_duplicates('ME')

In [43]:
HMM.columns = [c.replace(' ','_') for c in HMM.columns.values]

In [12]:
from tqdm.auto import tqdm
tqdm.pandas()

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [49]:
HMM = HMM.fillna('None')

In [52]:
HMM_DICT_LIST = []
for row in tqdm(HMM.itertuples()):
    gov = 100
    dhc = 100
    sym = 100
    iqv = 100
    if row.DESC != row.pri_spec:
        gov = fuzz.ratio(row.DESC, row.pri_spec)
    if row.DESC != row.Primary_Specialty.upper():
        dhc = fuzz.ratio(row.DESC, row.Primary_Specialty.upper())
    if row.DESC != row.PRCTR_DS_RPTD_PRMRY_SPCLT_DESC:
        sym = fuzz.ratio(row.DESC, row.PRCTR_DS_RPTD_PRMRY_SPCLT_DESC)
    if row.DESC != row.PRIMARY_SPEC:
        sym = fuzz.ratio(row.DESC, row.PRIMARY_SPEC)
    avg = (gov+dhc+sym+iqv)/4
    new_dict = {
        'ME': row.ME,
        'GOV':gov,
        'DHC': dhc,
        'SYM': sym,
        'IQV': iqv,
        'AVERAGE':avg
    }
    HMM_DICT_LIST.append(new_dict)

In [198]:
# yea = pd.DataFrame(HMM_DICT_LIST)
x = yea[yea.AVERAGE<70].ME

In [69]:
HMM[HMM.ME.isin(x)][['pri_spec','Primary_Specialty','PRIMARY_SPEC','PRCTR_DS_RPTD_PRMRY_SPCLT_DESC', 'DESC', 'DESC_SEC']]

,pri_spec,Primary_Specialty,PRIMARY_SPEC,PRCTR_DS_RPTD_PRMRY_SPCLT_DESC,DESC,DESC_SEC
92,PAIN MANAGEMENT,Pain Management,ANESTHESIOLOGY,PAIN MEDICINE,ANESTHESIOLOGY,OTHER SPECIALTY
144,CRITICAL CARE (INTENSIVISTS),Critical Care (Intensivists),GENERAL SURGERY,"SURGERY, GENERAL",GENERAL SURGERY,OTHER SPECIALTY
286,HOSPICE/PALLIATIVE CARE,Hospice and Palliative Care,None,INTERNAL MEDICINE,INTERNAL MEDICINE,OTHER SPECIALTY
294,NEPHROLOGY,Nephrology,NEPHROLOGY,NEPHROLOGY/RENAL MEDICINE,INTERNAL MEDICINE,OTHER SPECIALTY
326,EMERGENCY MEDICINE,Emergency Medicine,None,PHYSICAL MEDICINE & REHABILITATION,PHYSICAL MEDICINE AND REHABILITATION,OTHER SPECIALTY
...,...,...,...,...,...,...
114288,ORTHOPEDIC SURGERY,Surgery - Orthopedic Surgery,ORTHOPEDIC SURGERY,"SURGERY, ORTHOPEDIC",MUSCULOSKELETAL ONCOLOGY,PEDIATRIC ORTHOPEDICS
114860,ALLERGY/IMMUNOLOGY,Allergy/Immunology,ALLERGY & IMMUNOLOGY,ALLERGY & IMMUNOLOGY,PEDIATRICS,PEDIATRIC ALLERGY
114886,UROLOGY,Urology,PEDIATRICS,UROLOGY,PEDIATRICS,PEDIATRIC UROLOGY
114897,PEDIATRIC MEDICINE,Pediatric Medicine,PEDIATRIC ENDOCRINOLOGY,"ENDOCRINOLOGY, PEDIATRIC",OTHER SPECIALTY,PEDIATRIC ENDOCRINOLOGY


In [68]:
HMM = pd.merge(HMM, specs, left_on='SEC_SPEC_CD', right_on='SPEC_CD', suffixes = ['','_SEC'])

In [70]:
specs

,SPEC_CD,SPEC_ID,DESC
0,CLI,1945,CLINICAL INFORMATICS (INTERNAL MEDICINE)
1,IAN,1946,CLINICAL INFORMATICS (ANESTHESIOLOGY)
2,IRI,1947,INTERVENTIONAL RADIOLOGY - INTEGRATED
3,CIF,1948,CLINICAL INFORMATICS (FAMILY MEDICINE)
4,CIE,1949,CLINICAL INFORMATICS (EMERGENCY MEDICINE)
...,...,...,...
378,VIR,1884,VASCULAR AND INTERVENTIONAL RADIOLOGY
379,VM,1885,VASCULAR MEDICINE
380,VN,1886,VASCULAR NEUROLOGY
381,VS,1887,VASCULAR SURGERY


In [75]:
aamc_spec = pd.read_excel('../../Data/Measurement/AAMC_SPEC.xlsx', header=None)

In [82]:
aamc_spec = aamc_spec.rename(columns={0:'SPEC'})

In [106]:
spec_dicts = []
for row in aamc_spec.itertuples():
    splits = row.SPEC.split(',\xa0')
    if len(splits) == 2:
        spec = splits[1]
        sub_spec = splits[0]
    else:
        spec = row.SPEC
        sub_spec = 'None'
    new_dict = {
        'SPECIALTY':spec.upper(),
        'SUBSPECIALTY':sub_spec.upper()
    }
    spec_dicts.append(new_dict)

In [94]:
specs[specs.SPEC_CD.isin(ppd.PRIM_SPEC_CD)]

,SPEC_CD,SPEC_ID,DESC
0,CLI,1945,CLINICAL INFORMATICS (INTERNAL MEDICINE)
1,IAN,1946,CLINICAL INFORMATICS (ANESTHESIOLOGY)
2,IRI,1947,INTERVENTIONAL RADIOLOGY - INTEGRATED
3,CIF,1948,CLINICAL INFORMATICS (FAMILY MEDICINE)
4,CIE,1949,CLINICAL INFORMATICS (EMERGENCY MEDICINE)
...,...,...,...
378,VIR,1884,VASCULAR AND INTERVENTIONAL RADIOLOGY
379,VM,1885,VASCULAR MEDICINE
380,VN,1886,VASCULAR NEUROLOGY
381,VS,1887,VASCULAR SURGERY


In [107]:
aamc_specs = pd.DataFrame(spec_dicts)
# aamc_specs[aamc_specs.SPECIALTY=='Clinical Informatics']

In [108]:
ppd_spec_dicts = []
for row in specs.itertuples():
    if '(' in row.DESC:
        specialty = row.DESC.split('(')[1][:-1]
        subspecialty = row.DESC.split('(')[0][:-1]
    else:
        specialty = row.DESC
        subspecialty = 'None'
    new_dict = {
        'SPECIALTY':specialty.upper(),
        'SUBSPECIALTY':subspecialty.upper(),
        'SPEC_CD': row.SPEC_CD
    }
    ppd_spec_dicts.append(new_dict)
PPD_SPECS = pd.DataFrame(ppd_spec_dicts)

In [112]:
perf = pd.merge(PPD_SPECS, aamc_specs, on=['SPECIALTY','SUBSPECIALTY'])

In [140]:
PPD_SPECS[~PPD_SPECS.SPEC_CD.isin(perf.SPEC_CD)&(specs.SPEC_CD.isin(ppd.PRIM_SPEC_CD))][16:]

,SPECIALTY,SUBSPECIALTY,SPEC_CD
63,EMERGENCY MEDICINE,HOSPICE & PALLIATIVE MEDICINE,HPE
64,FAMILY MEDICINE,HOSPICE & PALLIATIVE MEDICINE,HPF
65,INTERNAL MEDICINE,HOSPICE & PALLIATIVE MEDICINE,HPI
66,OBSTETRICS & GYNECOLOGY,HOSPICE & PALLIATIVE MEDICINE,HPO
67,PEDIATRICS,HOSPICE & PALLIATIVE MEDICINE,HPP
...,...,...,...
378,VASCULAR AND INTERVENTIONAL RADIOLOGY,NONE,VIR
379,VASCULAR MEDICINE,NONE,VM
380,VASCULAR NEUROLOGY,NONE,VN
381,VASCULAR SURGERY,NONE,VS


In [116]:
aamc_specs[aamc_specs.SUBSPECIALTY == 'CLINICAL INFORMATICS']

,SPECIALTY,SUBSPECIALTY
27,PATHOLOGY,CLINICAL INFORMATICS
28,DIAGNOSTIC RADIOLOGY,CLINICAL INFORMATICS
29,ANESTHESIOLOGY,CLINICAL INFORMATICS
30,PREVENTIVE MEDICINE,CLINICAL INFORMATICS


In [142]:
HMM[HMM.PRIM_SPEC_CD == 'HPF'][['pri_spec','Primary_Specialty','PRIMARY_SPEC','PRCTR_DS_RPTD_PRMRY_SPCLT_DESC', 'DESC', 'DESC_SEC']]

,pri_spec,Primary_Specialty,PRIMARY_SPEC,PRCTR_DS_RPTD_PRMRY_SPCLT_DESC,DESC,DESC_SEC
98773,HOSPICE/PALLIATIVE CARE,Hospice and Palliative Care,HOSPICE & PALLIATIVE MED (FAMILY MED),PAIN MEDICINE,HOSPICE & PALLIATIVE MEDICINE (FAMILY MEDICINE),UNSPECIFIED
98774,HOSPICE/PALLIATIVE CARE,Hospice and Palliative Care,FAMILY MEDICINE,PAIN MEDICINE,HOSPICE & PALLIATIVE MEDICINE (FAMILY MEDICINE),UNSPECIFIED
98775,INTERVENTIONAL PAIN MANAGEMENT,Pain Management - Interventional Pain Management,HOSPICE & PALLIATIVE MED (FAMILY MED),PAIN MEDICINE,HOSPICE & PALLIATIVE MEDICINE (FAMILY MEDICINE),UNSPECIFIED
104433,HOSPICE/PALLIATIVE CARE,Hospice and Palliative Care,HOSPICE & PALLIATIVE MED (FAMILY MED),PAIN MEDICINE,HOSPICE & PALLIATIVE MEDICINE (FAMILY MEDICINE),FAMILY MEDICINE


In [118]:
iqvia[iqvia.ME=='9420110001']

,ROLE,PRIMARY_SPEC,SECONDARY_SPEC,PRIMARY_PROF_DESC,STATUS_DESC,TERTIARY_SPEC,COT_CLASSIFICATION,COT_FACILITY_TYPE,COT_SPECIALTY,ME


In [4]:
username = 'vigrose'
password_edw = 'Slytherin~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [5]:
ME_QUERY = \
        """
        SELECT DISTINCT
        P.PARTY_ID,
        P.KEY_VAL AS ME
        FROM
        AMAEDW.PARTY_KEY P
        WHERE
        P.KEY_TYPE_ID = 18
        AND
        P.ACTIVE_IND = 'Y'
        """
ME = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [6]:
QUERY = \
    f"""
    SELECT 
    PARTY_ID_FROM,
    CERTIF_TYPE_DESC, 
    ISS_DT, 
    EXP_DT, 
    OCCUR_TYPE_DESC, 
    DURATION_TYPE_DESC, 
    REC_DESC, 
    CERT_NM
    FROM
    AMAEDW.BRD_CERTIF
    WHERE
    THRU_DT is null
    """
BOARD = pd.read_sql(con=AMAEDW, sql=QUERY)

In [157]:
BOARD

,PARTY_ID_FROM,CERTIF_TYPE_DESC,ISS_DT,EXP_DT,OCCUR_TYPE_DESC,DURATION_TYPE_DESC,REC_DESC,CERT_NM
0,1798040,General,1938-11-12,None,Initial Certification,Lifetime,None,Dermatology
1,1798268,General,1971-04-30,None,Initial Certification,Lifetime,None,Psychiatry
2,1799689,General,2016-01-01,2025-12-31,Re-Certification,Time Limited,None,Clinical Molecular Genetics
3,1800138,Sub-specialty,1990-09-30,None,Initial Certification,Lifetime,None,Child & Adolescent Psychiatry
4,1800138,General,1988-10-30,None,Initial Certification,Lifetime,None,Psychiatry
...,...,...,...,...,...,...,...,...
3118168,2599497,General,2020-04-30,None,Re-Certification,Maintenance of Certification,None,Family Medicine
3118169,2203446,General,2021-01-05,None,Re-Certification,Maintenance of Certification,None,Family Medicine
3118170,2537755,General,2019-12-27,None,Re-Certification,Maintenance of Certification,None,Family Medicine
3118171,2485220,Sub-specialty,2018-04-28,2020-11-01,Re-Certification,Time Limited,None,Geriatric Medicine


In [8]:
ppd = pd.merge(ppd, ME, on='ME')
ppd = pd.merge(ppd, specs, left_on='PRIM_SPEC_CD', right_on='SPEC_CD')
ppd = pd.merge(ppd, specs, left_on='SEC_SPEC_CD', right_on='SPEC_CD', suffixes = ['','_SEC'])

In [9]:
PPD = pd.merge(BOARD, ppd, left_on='PARTY_ID_FROM', right_on='PARTY_ID')

In [10]:
def is_different(thing_1, thing_2):
    if thing_1 == thing_2:
        diff = False
    elif thing_1 in thing_2:
        diff = False
    elif thing_2 in thing_1:
        diff = False
    else:
        diff = True
    return(diff)

In [13]:
board_comps = []
for row in tqdm(PPD.itertuples()):
    match = 100
    if is_different(row.CERT_NM.upper(),row.DESC):
        match = fuzz.ratio(row.CERT_NM.upper(),row.DESC)
    new_dict = {
        'MATCH':match,
        'ME': row.ME,
        'CERT_NM': row.CERT_NM,
        'SPEC_1': row.DESC,
        'SPEC_2': row.DESC_SEC
    }
    board_comps.append(new_dict)

In [14]:
board_comps = pd.DataFrame(board_comps)

In [17]:
board_comps[(board_comps.SPEC_1!='UNSPECIFIED')]

,MATCH,ME,CERT_NM,SPEC_1,SPEC_2
1,21,01401980535,Internal Medicine,UNSPECIFIED,UNSPECIFIED
19,29,01606991505,Pediatrics,UNSPECIFIED,UNSPECIFIED
64,6,06501790751,Diagnostic Radiology,UNSPECIFIED,UNSPECIFIED
148,19,06201750341,Public Health & General Preventive Medicine,UNSPECIFIED,UNSPECIFIED
156,20,06201860189,Orthopaedic Surgery,UNSPECIFIED,UNSPECIFIED
...,...,...,...,...,...
2910222,24,03575091627,Anesthesiology,UNSPECIFIED,UNSPECIFIED
2910224,21,03575141519,Internal Medicine,UNSPECIFIED,UNSPECIFIED
2910225,31,65401182343,Family Medicine,UNSPECIFIED,UNSPECIFIED
2910226,31,66106170861,Family Medicine,UNSPECIFIED,UNSPECIFIED


In [32]:
board_comps[(board_comps.MATCH<30)&(~board_comps.SPEC_1.isin(['UNSPECIFIED','OTHER SPECIALTY']))].drop_duplicates('ME').to_csv('../../Data/Measurement/Specialty_Mismatch.csv')

In [193]:
IDK = ppd[ppd.ME.isin(board_comps[board_comps.SPEC_1.isin(['UNSPECIFIED','OTHER SPECIALTY'])].ME)].PARTY_ID

In [194]:
BOARD[BOARD.PARTY_ID_FROM.isin(IDK)]

,PARTY_ID_FROM,CERTIF_TYPE_DESC,ISS_DT,EXP_DT,OCCUR_TYPE_DESC,DURATION_TYPE_DESC,REC_DESC,CERT_NM
119,1843160,General,2002-08-20,2012-12-31,Initial Certification,Time Limited,None,Internal Medicine
160,1856293,General,2012-12-31,2013-12-31,Re-Certification,Time Limited,None,Obstetrics & Gynecology
162,1856293,General,2014-12-31,2015-12-31,Re-Certification,Time Limited,None,Obstetrics & Gynecology
164,1856293,General,2013-12-31,2014-12-31,Re-Certification,Time Limited,None,Obstetrics & Gynecology
165,1856293,General,2015-12-31,2016-12-31,Re-Certification,Time Limited,None,Obstetrics & Gynecology
...,...,...,...,...,...,...,...,...
3117411,10919021,Sub-specialty,2022-03-29,None,Initial Certification,Maintenance of Certification,None,Neonatal-Perinatal Medicine
3117419,6630020,General,2022-04-25,None,Re-Certification,Maintenance of Certification,None,Family Medicine
3117747,3732312,General,2020-10-07,None,Re-Certification,Maintenance of Certification,None,Family Medicine
3117750,6629003,General,2020-11-20,None,Re-Certification,Maintenance of Certification,None,Family Medicine


In [25]:
927485/1006083

0.9218772208654753